In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
gpus = tf.config.experimental.list_physical_devices(device_type="GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2024-01-03 12:02:33.563206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 12:02:35.735670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 12:02:35.755948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-

In [4]:
x = tf.Variable(tf.range(6, dtype=float), )
x

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=array([0., 1., 2., 3., 4., 5.], dtype=float32)>

In [5]:
y = tf.identity(x)
y

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0., 1., 2., 3., 4., 5.], dtype=float32)>

In [6]:
x[0].assign(-1.)

<tf.Variable 'UnreadVariable' shape=(6,) dtype=float32, numpy=array([-1.,  1.,  2.,  3.,  4.,  5.], dtype=float32)>

In [7]:
x

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=array([-1.,  1.,  2.,  3.,  4.,  5.], dtype=float32)>

In [9]:
z = tf.Variable(x) # New tensor
x.assign([0.] * 6)

<tf.Variable 'UnreadVariable' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0.], dtype=float32)>

In [10]:
z

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=array([-1.,  1.,  2.,  3.,  4.,  5.], dtype=float32)>

In [11]:
x

<tf.Variable 'Variable:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0.], dtype=float32)>

In [20]:
x = tf.Variable(1., shape=tf.TensorShape(None))
x

<tf.Variable 'Variable:0' shape=<unknown> dtype=float32, numpy=1.0>

In [21]:
x.assign([[2., 3.]])

<tf.Variable 'UnreadVariable' shape=<unknown> dtype=float32, numpy=array([[2., 3.]], dtype=float32)>

In [22]:
x.assign([[2., 3.], [4.0, 0.]])

<tf.Variable 'UnreadVariable' shape=<unknown> dtype=float32, numpy=
array([[2., 3.],
       [4., 0.]], dtype=float32)>

In [24]:
tf.TensorSpec.from_tensor(y)

TensorSpec(shape=(6,), dtype=tf.float32, name=None)

In [28]:
x = tf.Variable(1.0, )

In [29]:
x.assign(2.)
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>

In [30]:
x.assign_add(-2.)
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [31]:
x.assign_sub(4.)
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-4.0>

In [35]:
x.get_shape().as_list(), x.shape, x.read_value()

([], TensorShape([]), <tf.Tensor: shape=(), dtype=float32, numpy=-4.0>)

In [38]:
x.ref() == x.ref()

True

In [41]:
z = tf.Variable(x, name=x.name, shape=tf.TensorShape(None))
z.ref() == x.ref()

False

In [42]:
z.set_shape((4, 4))
z

<tf.Variable 'Variable:0:0' shape=(4, 4) dtype=float32, numpy=-4.0>

In [43]:
z.numpy()

-4.0

In [45]:
x.__div__(2.0)
x

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-4.0>

In [48]:
x = tf.Variable(tf.range(3.))
with tf.GradientTape() as tape:
    y = tf.reduce_sum(x**2) 

g = tape.gradient(y, x)
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 2., 4.], dtype=float32)>

In [50]:
x = tf.Variable(tf.range(3.))
with tf.GradientTape() as tape:
    y = 2*x

g = tape.gradient(y, x) # dy/dx = dy1/dx +  dy2/dx + dy3/dx
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

In [51]:
x = tf.Variable(tf.range(3.) + 1)
with tf.GradientTape() as tape:
    y = tf.math.log(x)

g = tape.gradient(y, x)
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.        , 0.5       , 0.33333334], dtype=float32)>

In [68]:
x = tf.Variable(tf.range(3.))
with tf.GradientTape() as tape:
    print(x, x[2])
    y = x + x[1] # [x1 + x2, 2*x2,  x2 + x3]
g = tape.gradient(y, x)
g

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 1., 2.], dtype=float32)> tf.Tensor(2.0, shape=(), dtype=float32)


<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 4., 1.], dtype=float32)>

In [70]:
u = tf.Variable(tf.range(3.), trainable=False)
with tf.GradientTape() as tape:
    y = u + u[1] # [x1 + x2, 2*x2,  x2 + x3]
g = tape.gradient(y, [u])
g

[None]

In [71]:
u = tf.Variable(tf.range(3.), trainable=False)
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    y = v**2
g = tape.gradient(y, [u, v])
g

[None, None]

In [73]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    y = v**2 + u 
g = tape.gradient(y, [u, v])
g

[<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 None]

In [72]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    y = v**2 + tf.stop_gradient(u) # dy/du = 0
g = tape.gradient(y, [u, v])
g

[None, None]

In [75]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    tape.watch(v)
    y = v + tf.stop_gradient(u) # dy/du = 0
g = tape.gradient(y, [u, v])
g

[None, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>]

In [76]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    tape.watch(v)
    y = v**2 + tf.stop_gradient(u) # dy/du = 0
g = tape.gradient(y, [u, v])
g

[None, <tf.Tensor: shape=(), dtype=float32, numpy=24.0>]

In [78]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    tape.watch(v)
    y = v**2 + tf.stop_gradient(u) # dy/du = 0
g = tape.gradient([y[0], y[1]], [u, v])
g

[None, None]

In [83]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    tape.watch(v)
    y = v**2 + tf.stop_gradient(u) # dy/du = 0
g = tape.gradient({"y": y[0]}, {"u": u, "v": v})
g

{'u': None, 'v': None}

In [85]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape() as tape:
    tape.watch(v)
    y = v**2 + tf.stop_gradient(u) # dy/du = 0
    z = 6*v
g = tape.gradient([z, y], {"u": u, "v": v})
g

{'u': None, 'v': <tf.Tensor: shape=(), dtype=float32, numpy=30.0>}

In [93]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape(watch_accessed_variables=False) as tape1, tf.GradientTape() as tape2:
        tape1.watch(v)
        # tape2.watch(v)
        y = v**2 + u 
        z = 6*v
g1 = tape1.gradient([z, y], {"u": u, "v": v})
g2 = tape2.gradient([z, y], {"u": u, "v": v})
print("g1 ", g1)
print("g2 ", g2)

g1  {'u': None, 'v': <tf.Tensor: shape=(), dtype=float32, numpy=30.0>}
g2  {'u': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, 'v': None}


In [124]:
u = tf.Variable(tf.range(3.), )
v = tf.constant(4.0)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(v)
    y = v**2 + u
    z = 6 * v
gu = tape.gradient([z, y], [u])
gv = tape.gradient([z, y], [v]) # Possible only when persistent is set to true
gu, gv

([<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=30.0>])

In [125]:
tape.gradient([y], [v]) 

[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>]

In [126]:
del tape

In [127]:
try:
    tape.gradient([y], [v]) 
except Exception as e:
    print("error ", e)

error  name 'tape' is not defined


In [105]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape(persistent=True) as tape:
    y = 2*u + 1
g = tape.gradient(y, u, )
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

In [109]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape(persistent=True) as tape:
    y = 2*u + 1
g = tape.gradient(y, u, output_gradients=tf.constant([1.0] * 3) ) # dl/ dy
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

In [123]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape(persistent=True) as tape:
    y = 2*u + 1
g = tape.gradient(y, u, output_gradients=tf.constant([2.0] * 3) ) # dl/ dy
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([4., 4., 4.], dtype=float32)>

In [114]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape() as tape:
    y = 2*u + 1
    z = u + 4
g = tape.gradient([y, z], u, output_gradients=[tf.constant([2.0] * 3), tf.constant([1.0] * 3)] ) # dl/ dy
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([5., 5., 5.], dtype=float32)>

In [119]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape() as tape:
    y = 2*u + 1
    z = u + 4
g = tape.gradient([y, z], u, output_gradients=[tf.constant([2.0] * 3), tf.constant([2.0] * 3),] ) # dl/ dy
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([6., 6., 6.], dtype=float32)>

In [120]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape() as tape:
    y = 2*u + 1
    z = u + 4
g = tape.gradient([y, z], u, output_gradients=[tf.constant([2.0] * 3), tf.constant([2.0] * 3), tf.constant([2.0] * 3), tf.constant([2.0] * 3)] ) # dl/ dy
g

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([6., 6., 6.], dtype=float32)>

In [122]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape() as tape:
    y = 2*u + 1
    print("y: ", y)
    z = y ** 2
g = tape.gradient(z, y)
g

y:  tf.Tensor([1. 3. 5.], shape=(3,), dtype=float32)


<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 2.,  6., 10.], dtype=float32)>

In [139]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape(persistent=True) as tape:
    y = 2*u + 1 + tf.reverse(u, (0,))
    z = y**2
    l = tf.reduce_sum(z)
g = tape.gradient(z, u)
jac = tape.jacobian(z, u)
g, jac

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([22., 24., 26.], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[12.,  0.,  6.],
        [ 0., 24.,  0.],
        [10.,  0., 20.]], dtype=float32)>)

In [140]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape(persistent=True) as tape:
    y = 2*u + 1 + tf.reverse(u, (0,))
    z = y**2
    l = tf.reduce_sum(z)
g = tape.gradient(l, u)
jac = tape.jacobian(l, u)
g, jac

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([22., 24., 26.], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([22., 24., 26.], dtype=float32)>)

In [144]:
u = tf.Variable(tf.range(3.), )
with tf.GradientTape() as tape1:
    with tf.GradientTape() as tape2:
        y = 2*u + 1 + tf.reverse(u, (0,))
        z = y**2
        l = tf.reduce_sum(z)
    g = tape2.gradient(l, u)
    hess = tape1.jacobian(g, u)
g, hess

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([22., 24., 26.], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[10.,  0.,  8.],
        [ 0., 18.,  0.],
        [ 8.,  0., 10.]], dtype=float32)>)

In [145]:
u = tf.Variable(1. )
v = tf.Variable(2. )
with tf.GradientTape() as tape:
    u.assign_add(v) # dy,dv = 0
    y = u**2 # y = u**2
g = tape.gradient(y, [u, v])
g

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]

In [146]:
u = tf.Variable(1. )
v = tf.Variable(2. )
with tf.GradientTape() as tape:
    u.assign_add(v) # dy,dv = 0
    y = u**2 # y = u**2
    u.assign_add(3.0)
g = tape.gradient(y, [u, v])
g

[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]

In [147]:
u = tf.Variable(1. )
with tf.GradientTape(persistent=True) as tape:
    y = u**2 
    tape.reset()
    z = u + 1
gy = tape.gradient(y, [u])
gz = tape.gradient(z, [u])
gy, gz

([None], [<tf.Tensor: shape=(), dtype=float32, numpy=1.0>])

In [150]:
u = tf.Variable(1. )
v = tf.Variable(1. )
with tf.GradientTape(persistent=True) as tape:
    y = u**2 + v
    z = 2 * y + v
gy = tape.gradient(y, [u, v])
gz = tape.gradient(z, [u, v])
gy, gz

([<tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.0>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=4.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=3.0>])

In [151]:
u = tf.Variable(1. )
v = tf.Variable(1. )
with tf.GradientTape(persistent=True) as tape:
    with tape.stop_recording():
        y = u**2 + v
    z = 2 * y + v
gy = tape.gradient(y, [u, v])
gz = tape.gradient(z, [u, v])
gy, gz

([None, None], [None, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>])

In [152]:
u = tf.Variable(1. )
v = tf.Variable(1. )
with tf.GradientTape(persistent=True) as tape:
    z = 2 * tf.stop_gradient(y) + v
gy = tape.gradient(y, [u, v])
gz = tape.gradient(z, [u, v])
gy, gz

([None, None], [None, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>])

In [156]:
tape.watched_variables()

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>,)

In [159]:
@tf.custom_gradient
def sin(x):
    def backward(dx):
        print("dx ", dx)
        return 2 * tf.cos(x) * dx
    return tf.sin(x), backward
u = tf.Variable(tf.experimental.numpy.pi / 3.)
with tf.GradientTape() as tape:
    y = sin(u)
g = tape.gradient(y, u)
g   

dx  tf.Tensor(1.0, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=0.99999994>

In [174]:
tf.cast(tf.math.exp(100.), dtype=tf.float64) / (1 + tf.cast(tf.math.exp(100.), dtype=tf.float64) ) 

<tf.Tensor: shape=(), dtype=float64, numpy=nan>

In [202]:
w, b = tf.Variable(tf.ones(shape=[3, 5])), tf.Variable(tf.ones(shape=(5, )))
@tf.custom_gradient
def linear(x):
    y = x @ w + b

    def backward(dy, variables): # compulsory variables args
        print("dx:  ", dy)
        print("weights:  ", variables)
        vw, vb = variables
        
        return dy @ tf.transpose(vw, (1, 0)), [tf.transpose(x, (1, 0)) @ dy, 2 * dy * tf.ones_like(vb)]
    return y, backward

x = tf.constant([1., 2., 3.], shape=(1, 3))
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  l = linear(x)
l

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[7., 7., 7., 7., 7.]], dtype=float32)>

In [203]:
tape.gradient(l, x)

dx:   tf.Tensor([[1. 1. 1. 1. 1.]], shape=(1, 5), dtype=float32)
weights:   [<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>]


<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[5., 5., 5.]], dtype=float32)>

In [204]:
tape.gradient(l, b)

dx:   tf.Tensor([[1. 1. 1. 1. 1.]], shape=(1, 5), dtype=float32)
weights:   [<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>]


<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[2., 2., 2., 2., 2.]], dtype=float32)>

In [206]:
tape.gradient(l, w)

dx:   tf.Tensor([[1. 1. 1. 1. 1.]], shape=(1, 5), dtype=float32)
weights:   [<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>]


<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.]], dtype=float32)>

In [207]:
w, b = tf.Variable(tf.ones(shape=[3, 5])), tf.Variable(tf.ones(shape=(5, )))
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  l = x @ w + b

In [208]:
tape.gradient(l, x)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[5., 5., 5.]], dtype=float32)>

In [209]:
tape.gradient(l, b)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>

In [210]:
tape.gradient(l, w)

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.]], dtype=float32)>